In [2]:
from eqcorrscan.utils.clustering import cluster
from eqcorrscan import Tribe
import numpy as np
from time import time

Matched-filter CPU is not compiled! Should be here: /home/jupyter_share/miniconda3/envs/seismo/lib/python3.8/site-packages/fast_matched_filter/lib/matched_filter_CPU.so


In [3]:
#set parameters
path = '/home/smocz/expand_redpy_new_files/templates/' #path to .tgz file
filename = 'Volcano_Rainier_Network_UW_Station_RCM_Channel_HHZ.tgz' #name of .tgz file
thresh_list = [0.2,0.3,0.4,0.5]

In [ ]:
#pull up tgz
t3 = time()
T = Tribe().read(f'{path}{filename}') #read the .tgz file

stream_list = [] #list of tuples. in tuple is (stream, template id)
for t in T:
    stream_tuple = (t.st,t.name)
    stream_list.append(stream_tuple)
t4 = time()
print(f'{t4-t3} s to get stream_list')


thresh_test=[]
for thresh in thresh_list:
    t0 = time()
    print('-----')
    print(f'testing theshold of: {thresh}')

    t1 = time()
    groups = cluster(template_list=stream_list, show=False,
                      corr_thresh=thresh, shift_len=15, cores=2) #corr_thresh is cross correlation threshold, shif_len is amount
    #it can shift by, cores is how many cores to use while running
    thresh_test.append(groups)
    #groups is a list of groups, each group is a list with the templates (tuple of stream and name) in it

    t2 = time()
    print(f'{t2-t0} s to cluster {filename}')

    print('# of groups',len(groups)) #number of clusters from EQcorrscan
    print('# of templates',len(T)) #number of template
    del(groups) #clear for less memory
#     break
    
print(f'{t2-t3} s to get all threshold values')

with open(f'/home/smocz/expand_redpy_new_files/eq_clusters{thresh_list[0]}-{thresh_list[-1]}.txt', 'w') as f: #save lists in txt file
    for groups in thresh_test:
        f.write("%s\n" % thresh_list[thresh_test.index(groups)])
        f.write("%s\n" % groups)

In [12]:
#read in lists of tuples from txt
thresh_test = [] 
with open('/home/smocz/expand_redpy_new_files/eq_clusters0.2-0.5.txt', 'r') as fp:
    print('oog')
    for #GET EACH LINE, APPEND EVERY OTHER (to account for titles) THEN USE LIST AS NORMAL

oog


In [10]:
print(thresh_test)

[]


In [9]:
# for groups in thresh_test:
groups = thresh_test[1]
print(' ')
print('#######################')
print(f'## THRESHOLD {thresh_list[thresh_test.index(groups)]}     ##')
print('#######################')
for gr in groups: #for each group
    if len(gr) > 1: #more than one tuple
        print('-----')
        print('index:',groups.index(gr))
        print('number of templates in group:',len(gr))
        for g in gr: #for each tuple
            print('*',g[1])
            g[0].plot(); #plot the first part (the stream)
#     break

IndexError: list index out of range

In [ ]:
for g in groups[57]:
    print('*',g[1])
    g[0].plot(); #plot the first part (the stream)

In [ ]:
print(len(groups[57]))

Figuring comparison between stations

I think there can be 3 outputs
1. conclusive - all (minsta?) stations have the the same grouping of clusters, everything is perfect. wow
2. inconclusive - doesn't make sense, no overlap that isn't just random
3. mostly conclusive - all (minsta?) stations SOME of the same grouping of clusters, enough that it's not random but isn't perfect


#### Now for the important part:
# How do we quantify these and process them?